Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [109]:
from random import choices, random, randint
import numpy as np
from copy import deepcopy

import lab9_lib

# GOAL: maximize fitness, minimize calls

In [110]:
fitness = lab9_lib.make_problem(5)
for n in range(20):
    ind = choices([0, 1], k=100)
    print(f"{''.join(str(g) for g in ind)}: {fitness(ind):.2%}")

print(fitness.calls)

1010101110001101011110000001000001010110001100000001001100010111001111100111110011011101110001000110: 11.70%
0110100101111111110000111110011010111101100110101101011000110101100000110101111111000100101110011100: 24.69%
0100100000000011110101010101001010000111111110111111001011101101010110110001010010110111000000010101: 22.91%
1100000101001010010011000010001010100001000000010101110101011100011101000001010001101111111010100010: 10.90%
1110011110100000100011101010000000100001000111000111000100111111011000100010110111001010110111100000: 10.78%
1101011001010111010101100011100000100010111000011001100011101001001101110100001001101111111110111000: 12.79%
1001010110110100001111011000010000011000100111001100001011100100101100010010011000101101101101000010: 11.00%
0101001100100000100000101100101100111100101011110011101010000110000101111011010011010110100000000011: 10.79%
1000100001001100100010011110010111000100111001110111110010001110000110010100100100001100010110111111: 10.89%
1110001110000000101

In [ ]:
Our code below

In [111]:
POPULATION_SIZE = 10
OFFSPRING_SIZE = 50
LOCI = 1000
TOURNAMENT_SIZE = 5
MUTATION_PROBABILITY = 0.25
BIT_FLIP_PROBABILITY = 0.05
NUM_GENERATION = 50

class Individual:
    def __init__(self):
        self.genotype = choices([0, 1], k=LOCI)
        self.fitness = float('-inf')


In [112]:
# Mutation / recombination or both
def tournament_selection(parents: list[Individual]) -> list[Individual]:
    return sorted(parents, key=lambda i: i.fitness, reverse=True)[:2]

def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, LOCI-1) 
    new_ind = Individual()
    new_ind.genotype = ind1.genotype[:cut_point] + ind2.genotype[cut_point:]
    assert len(new_ind.genotype) == LOCI
    return new_ind 

# This xover function returns a child whose genome 
# is created proportionally to the fitenss of parents
def uniform_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    p1 = ind1.fitness/(ind1.fitness + ind2.fitness)
    gene = [np.random.choice([ind1.genotype[i], ind2.genotype[i]], p=[p1, 1-p1]) for i in range(LOCI)]
    new_ind = Individual()
    new_ind.genotype = gene
    return new_ind

def mutate(parent: Individual) -> Individual:
    new_offspring = deepcopy(parent)
    for i in range(LOCI):
        if random() < BIT_FLIP_PROBABILITY:
            new_offspring.genotype[i] = int(not new_offspring.genotype[i])
    return new_offspring

def offspring_generation(parent1: Individual, parent2: Individual) -> Individual:
    if random() < MUTATION_PROBABILITY:
        # mutation
        return mutate(parent1)
    else:
        # cross_over
        return uniform_cut_xover(parent1, parent2)
    
def ea() -> Individual:
    # starting pouplation of POPULATION_SIZE individuals 
    population = [Individual() for _ in range(POPULATION_SIZE)]
    for p in population:
        p.fitness = fitness(p.genotype)        
    for _ in range(NUM_GENERATION):
        # best_old_one = best_one
        parents_idx = np.random.choice(range(len(population)), size=TOURNAMENT_SIZE, replace=False)
        parents = [population[idx] for idx in parents_idx]
        best_parents = tournament_selection(parents)
        for _ in range(OFFSPRING_SIZE):
            offspring = offspring_generation(best_parents[0], best_parents[1])
            offspring.fitness = fitness(offspring.genotype)
            population.extend([offspring])

        population.sort(key=lambda i: i.fitness, reverse=True)
        population = population[:POPULATION_SIZE] # always keep the first POPULATION_SIZE best individuals
    return population[0]


In [113]:
instance = [1, 2, 5, 10]
for k in instance:
    fitness = lab9_lib.make_problem(k)
    print(f'Best individual fitness: {ea().fitness}')
    print(f'Fintess calls: {fitness.calls}')    


KeyboardInterrupt: 